#  Numerical Implementation

## Preamble
* Define "vec" command for LaTeX $\newcommand{vec}[1]{\boldsymbol{#1}}$
* Define "mat" command for LaTeX $\newcommand{mat}[1]{\mathbb{#1}}$
* Define "set" command for LaTeX $\newcommand{set}[1]{\mathcal{#1}}$

## Discretized Grid Dynamics

### Spatial Discretization

The basic field equation to be solved for the grid dynamics is as follows:

\begin{equation}
    \ddot{s}
    + 2\left(\zeta\omega - \zeta^\prime\omega^\prime \nabla^2_\xi \right)\dot{s}
    + \left(\omega^2 - \omega^{\prime 2} \nabla^2_\xi \right)s 
    = 2\zeta\omega\dot{s}_s
    + \omega^2 s_s
\end{equation}

Our discretization of this equation will be cell-centered to match the discretization scheme of DPLR, which is where we ultimately expect to implement this algorithm. Furthermore, since we do not require high accuracy numerics or or strong conservation properties, we will use second order finite differences to approximate the spatial derivatives:

\begin{equation}
    \nabla^2_\xi s = s_{i-1,j} + s_{i+1,j} + s_{i,j-1} + s_{i,j+1} - 4s_{i,j} = -\left(4s_i - \sum_{j \in \mathcal{J}_i} s_j  \right)
\end{equation}

Where $\set{J}_i$ is the set cells neighboring the $i$-th cell in the mesh. Applying the discretization, we arrive at the following set of coupled, ordinary differential equations: 

\begin{equation}
    \ddot{s}_i
    + 2\left(\zeta\omega + 4\zeta^\prime\omega^\prime\right) \dot{s}_i
    + \left(\omega^2 + 4\omega^{\prime 2} \right) s_i 
    - 2\zeta^\prime\omega^\prime \sum \dot{s}_j
    - \omega^{\prime 2}\sum s_j
    = 2\zeta\omega\dot{s}_{s,i}
    + \omega^2 s_{s,i}
\end{equation}

Let $\vec{s}_i = [ s_i, \dot{s}_i ]^T $. With this notation, we can re-write our second order differential equation as system of first order equations. 

\begin{equation}
    \dot{\vec{S}}_i 
    + \left(\mat{U} + \mat{K} + 4\mat{K}^\prime \right)\vec{S}_i 
    - \sum \mat{K}^\prime \vec{S}_j
    = \mat{K}\vec{S}_s
\end{equation}
    
Where

\begin{align}
    \mat{U} & = \left[\begin{array}{cc}
        0 & 1 \\
        0 & 0 \\
    \end{array}\right] \\
    \mat{K} & = \left[\begin{array}{cc}
        0 & 0 \\
        \omega^2 & 2\zeta\omega \\
    \end{array}\right] \\
    \mat{K}^\prime & = \left[\begin{array}{cc}
        0 & 0 \\
        \omega^{\prime 2} & 2\zeta^\prime\omega^\prime \\
    \end{array}\right] \\
\end{align}


### Temporal Discretization
DLPR uses implicit time integration to circumvent the CFL restrictions associated with grids that have very fine near-wall spacing. The integration engine uses the 1st-order accurate Backward Euler scheme with the option of modifying the residual to achieve 2nd order accuracy. Since our grid dynamics equation is linear with constant coefficients, deriving the implicit time advancement equation is trivial.

\begin{equation}
    \frac{1}{\Delta t} \Delta \vec{S}_i
    + \left( \mat{U} + \mat{K} + 4\mat{K}^\prime \right) 
      \left( \vec{S}_i + \Delta\vec{S}_i \right)
    - \sum \mat{K}^\prime \left( \vec{S}_j + \Delta \vec{S}_j \right)
    = \mat{K} \left( \vec{S}_s + \Delta \vec{S}_s \right)
\end{equation}

\begin{equation}
    \left(\frac{\mat{I}}{\Delta t} + \mat{U} + \mat{K} + 4\mat{K}^\prime \right) \Delta \vec{S}_i 
    - \sum \mat{K}^\prime \Delta \vec{S}_j
    - \mat{K} \Delta \vec{S}_s
    = \vec{F}\left(\vec{S},\vec{S}_s\right)
\end{equation}

Where

\begin{equation}
    \vec{F}\left(\vec{S},\vec{S}_s\right) 
    = \mat{K}\vec{S}_s 
    + \sum \mat{K}^\prime \vec{S}_j    
    - \left(\mat{U} + \mat{K} - 4\mat{K}^\prime \right) \vec{S}_i
\end{equation}

In the case of a stationary shock where $\Delta\vec{S}_s = 0$, the equation above degenerates into a block-pentadiagonal linear system. However, while iterating to steady state, the shock will in fact move and change velocity depending on the local flow solution near the shock front. Therefore this is (one of) the terms whereby the linear solve for the grid dynamics couples into the linear solve for the flow dynamics.


## Discretized Fluid Dynamics

### Spatial Discretization
The fluid dynamics of for the problem are governed by the Navier Stokes equations, which can be written in integral form for an arbitrarily deforming control volume as follows: 

\begin{equation}
    \frac{d}{dt} \int_{V(t)} \vec{U} dV + \int_{\partial V(t)} \left( \mat{F} - \vec{U}\dot{\vec{x}}^T \right) \cdot d\vec{A} = 0
\end{equation}

Where, the flux matrix $\mat{F}$ is 

\begin{equation}
    \mat{F} = \left[ \vec{F}_z, \vec{F}_y, \vec{F}_z \right]
\end{equation}

A typical finite volume code will solve this conservation equation for each cell in the computational grid. Consider the $i$-th cell in a regular hexahedral mesh with the volume-average state $\vec{U}_i$. In this situation, the integral of the flux across the boundary of the control volume will be approximated using a midpoint quadrature on each of the six faces as follows:

\begin{equation} 
    \frac{d}{dt}\vec{U}_iV_i 
    + \sum_{j\in\set{J}_i} \left( \mat{F}_j - \vec{U}_j\dot{\vec{x}}_j^T \right) \cdot \vec{A}_j = 0
\end{equation} 

### Temporal Disretization

To apply the Backward Euler scheme to the preceding discrete system is fairly involved. In the general case, the flux matrix is a non-linear function of the reconstructed fluid state on either side of the cell face, $\vec{U}_j^L,\vec{U}_j^R$ and the reconstructed gradients, $\nabla\vec{U}_j^L,\nabla\vec{U}_j^R$. These reconstructions are in turn computed from a non-linear combination of the cell-averaged data in neighboring cells and may take into account the geometry of the grid. 